In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten,Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
import matplotlib.pyplot as plt
%matplotlib inline

Using Theano backend.


In [2]:
batch_size = 8
nb_classes = 9
#image_size=(218,303)
#input_shape=(3,218,303)
#image_size=(128,128)
#input_shape=(3,128,128)
image_size=(224,224)
input_shape=(3,224,224)
classes=["chicken","ostrich",'bluebird','finch','frog','salemander','cobra','bird','flamingo']

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '../data/cs231n_project/data/train',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
        '../data/cs231n_project/data/val',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 10177 images belonging to 9 classes.
Found 1328 images belonging to 9 classes.


In [4]:
nb_epoch = 30
nb_filters=32
kernel_size=(3,3)
pool_size=(2,2)

In [5]:
img=Input(input_shape)
resnet_model = ResNet50(weights=None,include_top=False)
print (resnet_model.output_shape)
middle=Input((2048,1,1))
x=Flatten()(middle)
x=Dense(128)(x)
x=Activation('relu')(x)
x=Dropout(0.5)(x)
x=Dense(nb_classes)(x)
x=Activation('softmax')(x)
top_model=Model(input=middle,output=x,name="topmodel")
model=Model(input=img,output=top_model(resnet_model(img)))

(None, 2048, None, None)


In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
resnet50 (Model)                 multiple              23587712    input_1[0][0]                    
____________________________________________________________________________________________________
topmodel (Model)                 (None, 9)             263433      resnet50[1][0]                   
Total params: 23,851,145
Trainable params: 23,798,025
Non-trainable params: 53,120
____________________________________________________________________________________________________
None


In [7]:
#first train all layers from scratch
trainscratch_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch,verbose=1)
trainscratch_score = model.evaluate_generator(test_generator, val_samples=100)
print('Test score:', trainscratch_score[0])
print('Test accuracy:', trainscratch_score[1])

00001	#include <Python.h>
00002	#include <iostream>
00003	#include "theano_mod_helper.h"
00004	#include <math.h>
00005	#include <numpy/arrayobject.h>
00006	#include <numpy/arrayscalars.h>
00007	#include <stdio.h>
00008	//////////////////////
00009	////  Support Code
00010	//////////////////////
00011	
00012	
00013	    extern "C"
00014	    {
00015	
00016	        void xerbla_(char*, void *);
00017	
00018	    /***********/
00019	    /* Level 1 */
00020	    /***********/
00021	
00022	    /* Single Precision */
00023	
00024	        void srot_(const int*, float *, const int*, float *, const int*, const float *, const float *);
00025	        void srotg_(float *,float *,float *,float *);
00026	        void srotm_( const int*, float *, const int*, float *, const int*, const float *);
00027	        void srotmg_(float *,float *,float *,const float *, float *);
00028	        void sswap_( const int*, float *, const int*, float *, const int*);
00029	        void scopy_( const int*, const float *, co

/usr/bin/ld: //usr/local/lib/libblas.a(sgemm.o): relocation R_X86_64_32 against `.rodata' can not be used when making a shared object; recompile with -fPIC
//usr/local/lib/libblas.a: error adding symbols: Bad value
collect2: error: ld returned 1 exit status



01197	            PyList_SET_ITEM(__ERROR, 0, err_type);
01198	            PyList_SET_ITEM(__ERROR, 1, err_msg);
01199	            PyList_SET_ITEM(__ERROR, 2, err_traceback);
01200	            {Py_XDECREF(old_err_type);}
01201	            {Py_XDECREF(old_err_msg);}
01202	            {Py_XDECREF(old_err_traceback);}
01203	        }
01204	        // The failure code is returned to index what code block failed.
01205	        return __failure;
01206	        
01207	        }
01208	    };
01209	    }
01210	    
01211	
01212	        static int __struct_compiled_op_b637c813fe0e6f963f19ec2f718a2b52_executor(__struct_compiled_op_b637c813fe0e6f963f19ec2f718a2b52* self) {
01213	            return self->run();
01214	        }
01215	
01216	        static void __struct_compiled_op_b637c813fe0e6f963f19ec2f718a2b52_destructor(void* executor, void* self) {
01217	            delete ((__struct_compiled_op_b637c813fe0e6f963f19ec2f718a2b52*)self);
01218	        }
01219	        
01220	//////////////////////


Exception: ('The following error happened while compiling the node', CorrMM{valid, (2, 2)}(IncSubtensor{InplaceSet;::, ::, int64:int64:, int64:int64:}.0, Subtensor{::, ::, ::int64, ::int64}.0), '\n', "Compilation failed (return status=1): /usr/bin/ld: //usr/local/lib/libblas.a(sgemm.o): relocation R_X86_64_32 against `.rodata' can not be used when making a shared object; recompile with -fPIC. //usr/local/lib/libblas.a: error adding symbols: Bad value. collect2: error: ld returned 1 exit status. ", '[CorrMM{valid, (2, 2)}(<TensorType(float32, 4D)>, <TensorType(float32, 4D)>)]')

In [ ]:
#now try with pretrain weights
resnet_model = ResNet50(weights='imagenet',include_top=False)
model=Model(input=img,output=top_model(resnet_model(img)))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print(model.summary())

In [ ]:
#first train all layers
trainall_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch,verbose=1)
trainall_score = model.evaluate_generator(test_generator, val_samples=100)
print('Test score:', trainall_score[0])
print('Test accuracy:', trainall_score[1])

In [ ]:
# now try with resnet frozen
for layer in resnet_model.layers:
    layer.trainable=False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print(model.summary())

In [ ]:
#now just train the top layers
traintop_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch,verbose=1)
traintop_score = model.evaluate_generator(test_generator, val_samples=100)
print('Test score:', traintop_score[0])
print('Test accuracy:', traintop_score[1])

In [ ]:
#now try spit training, training top first, then resnet
#resnet frozen
for layer in resnet_model.layers:
    layer.trainable=False

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print(model.summary())

In [ ]:
trainsplittop_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch/2,verbose=1)
trainsplittop_score = model.evaluate_generator(test_generator, val_samples=100)
print('Test score:', trainsplittop_score[0])
print('Test accuracy:', trainsplittop_score[1])

In [ ]:
top_model.save_weights("temp")

In [ ]:
#resnet frozen
for layer in resnet_model.layers:
    layer.trainable=True

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
top_model.load_weights("temp", by_name=True)
print(model.summary())

In [ ]:
model.load_weights("temp", by_name=True)

In [ ]:
trainsplitfull_hist=model.fit_generator(train_generator,samples_per_epoch=1000, 
                         validation_data=test_generator,nb_val_samples=100,
                         nb_epoch=nb_epoch/2,verbose=1)
trainsplitfull_score = model.evaluate_generator(test_generator, val_samples=10)
print('Test score:', trainsplitfull_score[0])
print('Test accuracy:', trainsplitfull_score[1])

In [ ]:
def legend(axis,title,score,acc,val_acc):
    axis.set_title(title)
    axis.text(labelx, labely,'Test score: {}'.format(score[0])
              ,transform=axis.transAxes)
    axis.text(labelx, labely*2, 'Test accuracy: {}  '.format(score[1])
              ,transform=axis.transAxes)
    axis.text(labelx, labely*3, 'Max Train acc: {}'.format(np.max(acc))
              ,transform=axis.transAxes)
    axis.text(labelx, labely*4, 'Max Val   acc: {} '.format(np.max(val_acc))
            ,transform=axis.transAxes)
    axis.plot(acc)
    axis.plot(val_acc)
f, axarr = plt.subplots(4,figsize=(12, 12))
labelx=.7
labely=.1

legend(axarr[0],'Train from scratch',trainscratch_score,trainscratch_hist.history['acc'],trainscratch_hist.history['val_acc'])
legend(axarr[1],'Train all layers',trainall_score,trainall_hist.history['acc'],trainall_hist.history['val_acc']) 
legend(axarr[2],'Train top layers',traintop_score,traintop_hist.history['acc'],traintop_hist.history['val_acc'])
legend(axarr[3],'Train top,then all layers',trainsplitfull_score,trainsplitfull_hist.history['acc'],trainsplitfull_hist.history['val_acc'])

